In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
import json
data = []
with open("../cs281_data/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json") as f:
    for l in f:
        data.append(json.loads(l))

In [3]:
business_list = []
    
for b in data:
    d = b.copy()
    
    # Dropping unconsidered features
    d.pop('hours')
    #d.pop('attributes')
    d.pop('neighborhoods')
    d.pop('type')
    d.pop('full_address')
    d.pop('name')    
    
    business_list.append(d)
    
business=pd.DataFrame(business_list)
# Changing the index
business = business.set_index('business_id')

business.head(20)

,attributes,categories,city,latitude,longitude,open,review_count,stars,state
business_id,,,,,,,,,
vcNAWiLM4dR7D2nwwJ7nCA,{u'By Appointment Only': True},"[Doctors, Health & Medical]",Phoenix,33.499313,-111.983758,True,9,3.5,AZ
UsFtqoBl7naz8AVUBZMjQQ,"{u'Happy Hour': True, u'Accepts Credit Cards':...",[Nightlife],Dravosburg,40.350519,-79.886930,True,4,3.5,PA
cE27W9VPgO88Qxe4ol6y_g,{u'Good for Kids': True},"[Active Life, Mini Golf, Golf]",Bethel Park,40.356896,-80.015910,False,5,2.5,PA
HZdLhv6COCleJMo7nPl-RA,{},"[Shopping, Home Services, Internet Service Pro...",Pittsburgh,40.357620,-80.059980,True,3,3.5,PA
mVHrayjG3uZ_RLHkLj-AMg,"{u'Alcohol': u'full_bar', u'Noise Level': u'av...","[Bars, American (New), Nightlife, Lounges, Res...",Braddock,40.408735,-79.866351,True,11,4.5,PA
KayYbHCt-RkbGcPdGOThNg,"{u'Alcohol': u'full_bar', u'Noise Level': u'lo...","[Bars, American (Traditional), Nightlife, Rest...",Carnegie,40.415517,-80.067534,True,15,4.0,PA
b12U9TFESStdy7CsTtcOeg,{},"[Auto Repair, Automotive]",Carnegie,40.394588,-80.084454,True,5,1.5,PA
Sktj1eHQFuVa-M4bgnEh8g,{u'Good for Kids': True},"[Active Life, Mini Golf]",Carnegie,40.405404,-80.076267,True,4,4.0,PA
3ZVKmuK2l7uXPE6lXY4Dbg,{},"[Home Services, Contractors]",Carnegie,40.406324,-80.090357,True,3,2.5,PA


In [4]:
Lasvegas = np.array(business.city[business.city=='Las Vegas'].index.values,dtype=str)

In [6]:
np.save("temp/lasvegas_id",Lasvegas)

In [7]:
import json
review = []
with open("../cs281_data/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json") as f:
    for r in f:
        review.append(json.loads(r))

In [8]:
rest = np.load('temp/restaurants_index.npy')

In [9]:
business_ids = [r['business_id'] for r in review]

In [10]:
combined_review_dic = dict.fromkeys(list(set(business_ids)))
business_ids_tuples = zip(range(len(business_ids)),business_ids)
for tup in business_ids_tuples:
    if combined_review_dic[tup[1]] is None:
        combined_review_dic[tup[1]] = [review[tup[0]]['text'],[review[tup[0]]['stars']]]
    else:
        combined_review_dic[tup[1]][0] = " ".join([combined_review_dic[tup[1]][0], review[tup[0]]['text']])
        (combined_review_dic[tup[1]][1]).append(review[tup[0]]['stars'])

In [11]:
restaurant_reviews_lasvegas = {}
for r in rest:
    if r in Lasvegas:
        restaurant_reviews_lasvegas[r] =  combined_review_dic[r]

In [13]:
len(restaurant_reviews_lasvegas)

3822

In [14]:
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")
def get_parts(thetext,seperate_w_adj):
    import itertools
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    if seperate_w_adj:
        return list(itertools.chain.from_iterable(nouns2)), list(itertools.chain.from_iterable(descriptives2))
    else:
        return list(itertools.chain.from_iterable(nouns2+descriptives2))

In [17]:
%%time
lasvegas_reviews_wandaj = {}
it = 0
for k in restaurant_reviews_lasvegas.keys():
    if it % 50 == 0:
        print it
    lasvegas_reviews_wandaj[k] = [get_parts(restaurant_reviews_lasvegas[k][0],True)]
    it+=1

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
CPU times: user 1h 32min 50s, sys: 1min 6s, total: 1h 33min 57s
Wall time: 1h 36min 5s


In [18]:
with open('temp/lasvegas_word_adjective.json', 'w') as fp:
    json.dump(lasvegas_reviews_wandaj, fp)

In [19]:
vocab = []
for i in lasvegas_reviews_wandaj.keys():
    vocab.extend(lasvegas_reviews_wandaj[i][0][0])

In [21]:
import collections
lv_vocab10 = []
d = collections.Counter(vocab)
for w in vocab:
    if d[w]>=10:
        lv_vocab10.append(w)

In [25]:
len(lv_vocab10)

10364

In [24]:
lv_vocab10 = list(set(lv_vocab10)) 

In [27]:
reduced_lv = {}
it = 0
for (k,v) in lasvegas_reviews_wandaj.items():
    if it % 50 == 0:
        print it
    new = []
    for w in v[0][0]:
        if w in lv_vocab10:
            new.append(w)
    reduced_lv[k] = new
    it+=1

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800


In [28]:
lv_word_to_index  = dict(zip(lv_vocab10,range(len(lv_vocab10))))
lv_index_to_word = dict(zip(range(len(lv_vocab10)),lv_vocab10))
lv_bid_to_index = dict(zip(reduced_lv.keys(),range(len(reduced_lv.keys()))))
lv_index_to_bid = dict(zip(range(len(reduced_lv.keys())),reduced_lv.keys()))

In [29]:
from scipy import sparse

nonzero_data = []
rows_s = []
cols_s = []

for k in reduced_lv.keys():
    counter = collections.Counter(reduced_lv[k])
    nonzero_data += counter.values()
    rows_s += [lv_bid_to_index[k]]*len(counter.values())
    cols_s += [lv_word_to_index[ck] for ck in counter.keys()]

sparse_mat = sparse.csc_matrix((nonzero_data,(rows_s,cols_s)),shape = (len(lv_bid_to_index),len(lv_word_to_index)))

In [30]:
dtm_lv = sparse_mat.toarray()

In [33]:
np.save("temp/lasvegas_dtm",dtm_lv)